In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
import P7_DataBreed

**Execute following cell in order to update object version with no change in attributes**

* A backup instance of object is created. This backup instance is empty.
* If object instance is not defined then it is read from dmped file.
* object instance is copied back into backup instance
* Backup instance is deleted

In [3]:
new_attribute=None

import p5_util
import P7_DataBreed

if new_attribute is None :
    is_saved = True
    filename='./data/oP7_DataBreed.dump'
    oP7_DataBreed_save = P7_DataBreed.P7_DataBreed()
    try: 
        oP7_DataBreed
    except NameError:
        print('*** INFO : oP7_DataBreed is not defined; loading...')
        oP7_DataBreed = p5_util.object_load(filename)
        is_saved = False
    oP7_DataBreed_save.copy(oP7_DataBreed)
    oP7_DataBreed = P7_DataBreed.P7_DataBreed()
    oP7_DataBreed.copy(oP7_DataBreed_save)
    del(oP7_DataBreed_save)
    oP7_DataBreed.show()
    is_saved=True
    if is_saved is True:
        print('*** INFO : oP7_DataBreed is saved')
        p5_util.object_dump(oP7_DataBreed,filename)


*** INFO : oP7_DataBreed is not defined; loading...
p5_util.object_load : fileName= ./data/oP7_DataBreed.dump

 
Path to data directory ........ : ./data/EasyImages
Number of breeds .............. : 12
Total number of images ........ : 913
Standard images size .......... : (200, 200)
SIFT Image descriptors count .. : 3543
Sampling : breeds count ....... : 10
Sampling : images per breed ... : 5
X train size .................. : (3188, 19)
y train size .................. : (3188, 7)
X test size ................... : (355, 19)
y test size ................... : (355, 7)
Clusters models  .............. : dict_keys(['GMM'])
Current cluster model  ........ : GMM
Bag of features dataframe ..... : (3543, 19)
Labels from dataset ........... : (3543, 7)
Number of breeds .............. : 7
Image splitted ................ : True
Key point descriptors ......... : (111260, 128)
Classifier name ............... : Bernoulli NB
Supported classifiers ......... : ['Bernoulli NB']

*** INFO : oP7_DataBreed 

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D

my_VGG16 = Sequential()  # Création d'un réseau de neurones vide 

# Ajout de la première couche de convolution, suivie d'une couche ReLU
my_VGG16.add(Conv2D(64, (3, 3), input_shape=(200, 200, 3), padding='same', activation='relu'))

# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
my_VGG16.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

# Ajout de la première couche de pooling
my_VGG16.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

from keras.layers import Flatten, Dense

my_VGG16.add(Flatten())  # Conversion des matrices 3D en vecteur 1D

# Ajout de la première couche fully-connected, suivie d'une couche ReLU
my_VGG16.add(Dense(7, activation='relu'))

# Ajout de la deuxième couche fully-connected, suivie d'une couche ReLU
my_VGG16.add(Dense(7, activation='relu'))

# Ajout de la dernière couche fully-connected qui permet de classifier
my_VGG16.add(Dense(7, activation='softmax'))

In [15]:
from keras import optimizers
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

my_VGG16.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


In [44]:
import numpy as np

pil_image = oP7_DataBreed.read_image(dirbreed, imagefilename)
breedname = dirbreed.split('-')[1]
df = pd.DataFrame([[breedname, pil_image]], columns=['breed','image'])
df.sample()

,breed,image
0,Doberman,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [26]:
dirbreed,imagefilename

('n02107142-Doberman', 'n02107142_4725.jpg')

In [71]:
import pandas as pd
import P7_DataBreed
dict_image = dict()
label = 0
df = pd.DataFrame()
new_breedname = str()
for dirbreed, list_imagefilename in oP7_DataBreed._dict_data.items():
    for imagefilename in list_imagefilename :
        pil_image = oP7_DataBreed.read_image(dirbreed, imagefilename)
        breedname = dirbreed.split('-')[1]
        df = df.append(pd.DataFrame([[breedname, label,pil_image]], columns=['breed','label','image']))
        pil_image.close()
    label +=1
print("Image count = "+str(len(df)))
df.reset_index(drop=True, inplace=True)
df.sample()


Image count = 913


,breed,label,image
616,Yorkshire_terrier,8,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [72]:
df['image'] = df['image'].apply(lambda x : x.resize((200,200)))

ValueError: Operation on closed image

In [73]:
df.iloc[0]['image']

ValueError: Operation on closed image

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x333 at 0x7EFE49406CF8>

In [63]:
from  sklearn import model_selection
X_train, X_test, y_train,  y_test = model_selection.train_test_split(df.index,df['label'],test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((730,), (730,), (183,), (183,))

In [67]:
df.iloc[:]

,breed,label,image
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...
0,Doberman,0,<PIL.Image.Image image mode=RGB size=200x200 a...


In [56]:
my_VGG16.fit(X_train, y_train)

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (730, 1)